# H3 Intro

In [ ]:
import h3
import pandas as pd
import folium
import branca.colormap as cm


#### Set Coordinates and Resolution

In [ ]:

lat = 39.73632141579045
lng = -104.9897890191563
resolution = 10

#### Convert Coordinates to H3

In [ ]:
h3_index = h3.geo_to_h3(lat, lng, resolution)
print(f"H3 Index: {h3_index}")

#### Convert H3 to Coordinates

In [ ]:
lat_lng = h3.h3_to_geo(h3_index)
print(f"Latitude/Longitude: {lat_lng}")


#### Convert H3 to Geometry Boundary

In [ ]:
boundary = h3.h3_to_geo_boundary(h3_index)
print(f"Boundary: {boundary}")

#### Map the Boundary

In [ ]:
# Create a map
m = folium.Map(location=[lat, lng], zoom_start=16)

# Add the H3 cell to the map
folium.Polygon(locations=boundary, color='blue', weight=5).add_to(m)

# Save the map
m


## Real Data: 311 Calls about illegal parking in Denver, Colorado

#### Import and filter data

In [ ]:
denver_311 = pd.read_csv("311_service_data_2023.csv")
denver_311 = denver_311[denver_311['Case Summary']=='Pothole']
denver_311 = denver_311.dropna(subset=['Latitude', 'Longitude'])
denver_311.shape

#### Create function to parse each row to create H3 index string and geometry boundaries 

In [ ]:
def compute_h3_and_boundaries(row, resolution=8):
    lat = row['Latitude']
    lng = row['Longitude']
    h3_index = h3.geo_to_h3(lat, lng, resolution)
    boundaries = h3.h3_to_geo_boundary(h3_index)
    return pd.Series([h3_index, boundaries])

#### Implement function and check the results

In [ ]:
denver_311[['H3_Index', 'Boundaries']] = denver_311.apply(compute_h3_and_boundaries, axis=1)
denver_311[['H3_Index', 'Boundaries']]

#### Aggregate by H3 index to get count of pothole reports

In [ ]:
# Aggregate the count of each H3 index
denver_311_counts = denver_311.groupby('H3_Index').size().reset_index(name='Count')
denver_311_counts


#### Merge in boundaries to the pothole counts

In [ ]:
# Merge the boundaries back with the counts
denver_311_counts = denver_311_counts.merge(
    denver_311[['H3_Index', 'Boundaries']].drop_duplicates(), on='H3_Index', how='left'
)
denver_311_counts

#### Visualize on a map

In [ ]:

m = folium.Map(location=[lat, lng], zoom_start=11, tiles='cartodbpositron')

# Create a color map using shades of red
colormap = cm.LinearColormap(colors=['#ffcccc', '#ff0000'], vmin=denver_311_counts['Count'].min(), vmax=80)

# Add the H3 cells to the map with color based on count
for _, row in denver_311_counts.iterrows():
    boundaries = row['Boundaries']
    count = row['Count']
    color = colormap(count)
    folium.Polygon(
        locations=boundaries,
        color=color,
        weight=1,
        fill=True,
        fill_opacity=0.7,
        popup=f'Count: {count}'
    ).add_to(m)

# Add the colormap to the map
colormap.add_to(m)


# Display the map
m
